In [2]:
!source ~/.bashrc

In [19]:
import wandb
import os
import json
from tqdm import tqdm
import numpy as np
import datetime

# Initialize the wandb API
api = wandb.Api()

# List all runs in the werewolf project
runs = list(api.runs("ohadrubin/werewolf"))


# Create a dataframe with all runs
import pandas as pd

runs_data = []
for run in runs:
    runs_data.append({
        "Run ID": run.id,
        "Name": run.name,
        "Status": run.state
    })

runs_df = pd.DataFrame(runs_data)
print(f"Found {len(runs)} runs in the werewolf project:")
display(runs_df)


# Create a dictionary to store files for each run
run_files = {}

# Iterate through finished runs and get their files
for _, run_row in tqdm(runs_df.iterrows(), total=len(runs_df), desc="Fetching files"):
    run_id = run_row["Run ID"]
    run_name = run_row["Name"]
    
    try:
        # Get the run object
        run = api.run(f"ohadrubin/werewolf/{run_id}")
        
        # Get files for this run
        files = [file.name for file in run.files()]
        # Download the content of each file
        file_contents = {}
        for file in run.files():
            if "results" not in file.name:
                continue
            try:
                content = file.download(replace=True)
                with open(file.name, 'rb') as f:
                    file_contents[file.name] = f.read()
            except Exception as e:
                print(f"Error downloading file {file.name}: {str(e)}")
        run_files[run_id] = {
            "files": file_contents,
        }
    except Exception as e:
        print(f"Error fetching files for run {run_id} ({run_name}): {str(e)}")



Found 24 runs in the werewolf project:


,Run ID,Name,Status
0,2z2sofpk,_t=Defense_nc=2_tp=True_s=small_ty=whisper,finished
1,87ebv6r3,_t=Accusation_nc=2_tp=True_s=small_ty=whisper,finished
2,ouloqpxg,_t=Call_for_Action_nc=2_tp=True_s=small_ty=whi...,finished
3,ogtv1kt0,_t=Evidence_nc=2_tp=True_s=small_ty=whisper,finished
4,8x3addbc,_t=Identity_Declaration_nc=2_tp=True_s=small_t...,finished
5,it4w76rt,_t=Interrogation_nc=2_tp=True_s=small_ty=whisper,finished
6,jwx42d0p,_t=Accusation_nc=2_tp=False_s=small_ty=whisper,finished
7,xaizshi8,_t=Call_for_Action_nc=2_tp=False_s=small_ty=wh...,finished
8,xkdjjoaz,_t=Defense_nc=2_tp=False_s=small_ty=whisper,finished
9,l84h5c7m,_t=Evidence_nc=2_tp=False_s=small_ty=whisper,finished


Fetching files: 100%|██████████| 24/24 [00:53<00:00,  2.21s/it]


In [21]:
# Create a mapping dictionary of run_id to run_name
run_id_to_name = dict(zip(runs_df["Run ID"], runs_df["Name"]))
run_id_to_name

{'2z2sofpk': '_t=Defense_nc=2_tp=True_s=small_ty=whisper',
 '87ebv6r3': '_t=Accusation_nc=2_tp=True_s=small_ty=whisper',
 'ouloqpxg': '_t=Call_for_Action_nc=2_tp=True_s=small_ty=whisper',
 'ogtv1kt0': '_t=Evidence_nc=2_tp=True_s=small_ty=whisper',
 '8x3addbc': '_t=Identity_Declaration_nc=2_tp=True_s=small_ty=whisper',
 'it4w76rt': '_t=Interrogation_nc=2_tp=True_s=small_ty=whisper',
 'jwx42d0p': '_t=Accusation_nc=2_tp=False_s=small_ty=whisper',
 'xaizshi8': '_t=Call_for_Action_nc=2_tp=False_s=small_ty=whisper',
 'xkdjjoaz': '_t=Defense_nc=2_tp=False_s=small_ty=whisper',
 'l84h5c7m': '_t=Evidence_nc=2_tp=False_s=small_ty=whisper',
 '99aqacni': '_t=multitask_nc=2_tp=True_s=small_ty=whisper',
 'xteepj3u': '_t=multitask_nc=2_tp=False_s=small_ty=whisper',
 'v1irg5yd': '_t=Identity_Declaration_nc=2_tp=False_s=small_ty=whisper',
 'd3rpcukj': '_t=Interrogation_nc=2_tp=False_s=small_ty=whisper',
 '49obucn4': '_t=multitask_nc=6_tp=True_s=small_ty=whisper',
 'nd33p8tl': '_t=multitask_nc=6_tp=True_

In [23]:
{k:{**v, "run_name": run_id_to_name.get(k, "Unknown")} for k,v in run_files.items()}

{'2z2sofpk': {'files': {'results_test.json': b'{\n  "loss": 0.7255098734583173,\n  "f1": 0.25363825363825365,\n  "precision": 0.27232142857142855,\n  "recall": 0.23735408560311283,\n  "accuracy": 0.7996651785714286,\n  "report": "              precision    recall  f1-score   support\\n\\n           0       0.88      0.89      0.88      3070\\n           1       0.27      0.24      0.25       514\\n\\n    accuracy                           0.80      3584\\n   macro avg       0.57      0.57      0.57      3584\\nweighted avg       0.79      0.80      0.79      3584\\n"\n}',
   'results_val.json': b'{\n  "loss": 0.7173255218399895,\n  "f1": 0.3128834355828221,\n  "precision": 0.36823104693140796,\n  "recall": 0.272,\n  "accuracy": 0.8055555555555556,\n  "report": "              precision    recall  f1-score   support\\n\\n           0       0.87      0.91      0.89      1929\\n           1       0.37      0.27      0.31       375\\n\\n    accuracy                           0.81      2304\